In [34]:
import laspy as lp
import numpy as np
import matplotlib.pyplot as plt


def read_las_file(las_path: str):
    try:
        return lp.read(las_path)
    except Exception as e:
        print(f"Error reading {las_path}: {e}")
        return None

def normalize_points(points, eps= 1e-8):
    if not isinstance(points, np.ndarray) or points.shape[1] != 3:
        raise ValueError("points must be a numpy array with shape (n, 3).")
    points_ = points.copy()

    x_min, x_max = np.min(points_[:,0]), np.max(points_[:,0])
    y_min, y_max = np.min(points_[:,1]), np.max(points_[:,1])
    z_min, z_max = np.min(points_[:,2]), np.max(points_[:,2])

    points_[:,0] = (points_[:,0] - x_min) / (x_max - x_min + eps)
    points_[:,1] = (points_[:,1] - y_min) / (y_max - y_min + eps)
    points_[:,2] = (points_[:,2] - z_min) / (z_max - z_min + eps)
    
    return points_, x_min, x_max, y_min, y_max, z_min, z_max  
   
def normalize_colors(colors):
    rgba = colors - colors.min(axis=0)
    rgba = rgba / rgba.max(axis=0)
    return rgba #(rgba * 255).astype(np.uint8)


rasantes = {
    'km11': 2300.382,  
    'km12': 2300.345,   
    'km13': 2300.372,
    '3a': 2300.305,
    '2a': 2300.700,
    'pam2': 2300.618  
}

salmueras = {
    'km11': 2300.866,
    'km12': 2300.582,
    'km13': 2300.647,
    'pam2': 2302.185,
    '2a': 2301.129,
    '3a': 2300.978
}

pozas = {
    "km11": [
        (565069.970, 7403193.413, 2300.565),
        (565191.968, 7403283.869, 2300.542),
        (565195.699, 7403102.580, 2300.527)
    ],
    "km12": [
        (565328.392, 7403116.663, 2300.408),
        (565334.326, 7403279.183, 2300.385),
        (565451.299, 7403199.079, 2300.392)
    ],
    "km13": [
        (565613.237, 7403203.757, 2300.486),
        (565724.189, 7403120.616, 2300.371),
        (565719.644, 7403294.142, 2300.385)
    ],
    "pam2": [
        (558854.768, 7394807.683, 2301.750),
        (558854.742, 7394746.032, 2301.614),
        (558896.341, 7394774.391, 2301.863)
    ],
    "2a": [
        (559312.399, 7394750.575, 2300.889),
        (559258.428, 7394683.524, 2300.824),
        (559257.914, 7394807.991, 2300.878)
    ],
    "3a": [
        (559421.390, 7394768.078, 2300.679),
        (559527.662, 7394863.970, 2300.698),
        (559533.954, 7394694.214, 2300.640)
    ]
}


DATE = 250330
POZA = '3a'
PATH_LAS = f'data/{DATE}_100m_7ms_100khz_plena_luz/pozas/{POZA}_100m_7ms_100khz_plena-luz_0_0.las' 
LIMIT = -1

rasante = rasantes.get(POZA, None)
c_salmuera = salmueras.get(POZA, None)
i_puntos = pozas.get(POZA,None)

print(f'{POZA}\nRasante: {rasante}\nCota Salmuera: {c_salmuera}\nPuntos Control: {i_puntos}')

3a
Rasante: 2300.305
Cota Salmuera: 2300.978
Puntos Control: [(559421.39, 7394768.078, 2300.679), (559527.662, 7394863.97, 2300.698), (559533.954, 7394694.214, 2300.64)]


In [35]:
las = read_las_file(PATH_LAS)

points = np.vstack((las.x, las.y, las.z)).transpose()
data_points = np.hstack((points,
                        np.expand_dims(las.return_number, -1),
                        np.expand_dims(las.number_of_returns, -1)))

first = data_points[(data_points[:,-2] == 1) & (data_points[:,-1] == 2)][:, :3]
one_return = data_points[(data_points[:, -1] == 1)][:, :3]

x, y, z = one_return[:,0], one_return[:,1], one_return[:,2] 


In [36]:
import numpy as np
from scipy.spatial import KDTree

points = np.vstack((x, y)).transpose()
kdtree = KDTree(points)
radius = 0.5

for xi, yi, cota_sal_sqm in i_puntos:
    point_of_interest = np.array([xi, yi])
    indices = kdtree.query_ball_point(point_of_interest, radius)
    cota_sal = np.mean(z[indices])
    print(f'Cota sal SQM: {cota_sal_sqm}\nCota lidar {cota_sal:.3f}\n')




Cota sal SQM: 2300.679
Cota lidar 2300.674

Cota sal SQM: 2300.698
Cota lidar 2300.688

Cota sal SQM: 2300.64
Cota lidar 2300.705

